In [1]:
import os
import glob
import random
import tqdm

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import PIL.Image as Image

In [3]:
import tensorflow as tf
tf.__version__

'2.13.0'

In [4]:
%cd /content/drive/MyDrive/Datasets/MIPI_tetra/train

[Errno 2] No such file or directory: '/content/drive/MyDrive/Datasets/MIPI_tetra/train'
/Users/bw/Code/lightning/src


In [5]:
%ls -la

total 92376
drwxr-xr-x@ 35 bw  staff     1120  2 13 21:56 ./
drwxr-xr-x@ 12 bw  staff      384  2  4 00:26 ../
-rw-r--r--@  1 bw  staff     6148  2 13 15:49 .DS_Store
drwxr-xr-x@ 10 bw  staff      320  2 13 18:25 .ipynb_checkpoints/
drwxr-xr-x@  4 bw  staff      128  2  6 00:29 __pycache__/
-rw-r--r--@  1 bw  staff     6080  1 30 17:08 bp_correction.py
-rw-r--r--@  1 bw  staff     2760  1 30 23:08 cure_input.py
-rw-r--r--@  1 bw  staff     9264  2 13 00:40 inference.py
-rw-r--r--@  1 bw  staff    18653  2  4 00:25 inference_gogo.py
-rw-r--r--@  1 bw  staff  9064309  1 26 22:34 inference_notebook.ipynb
-rw-r--r--@  1 bw  staff     8438  2  7 00:52 inference_single.py
drwxr-xr-x@  2 bw  staff       64  2 13 15:41 mit_pairs/
-rw-r--r--@  1 bw  staff   357582  2  5 02:56 model.png
drwxr-xr-x@  4 bw  staff      128  2  5 02:31 model_dir/
-rw-r--r--@  1 bw  staff    10076  1 29 00:17 mydataset.py
-rw-r--r--@  1 bw  staff     4782  2  6  2023 myloss.py
-rw-r--r--@  1 bw  staff     2650  1 27 

In [6]:
cwd = os.getcwd()
cwd

'/Users/bw/Code/lightning/src'

# MIT dataset loading

In [22]:
paths = ['/Users/bw/Dataset/MIT/images/train']
paths = ['/Users/bw/Dataset/MIT/images/val',
        '/Users/bw/Dataset/MIT/images/test' ]
paths = ['/Users/bw/Dataset/MIT/images/viz10']

In [23]:
%pwd

'/Users/bw/Code/lightning/src'

In [24]:
fnames = []

for path in paths:
    print(path)
    fnames += glob.glob(os.path.join(path, '**/*.png'), recursive=True)
print(len(fnames))

/Users/bw/Dataset/MIT/images/viz10
10


In [25]:
fnames[:10]

['/Users/bw/Dataset/MIT/images/viz10/000008.png',
 '/Users/bw/Dataset/MIT/images/viz10/000009.png',
 '/Users/bw/Dataset/MIT/images/viz10/000007.png',
 '/Users/bw/Dataset/MIT/images/viz10/000006.png',
 '/Users/bw/Dataset/MIT/images/viz10/000004.png',
 '/Users/bw/Dataset/MIT/images/viz10/000010.png',
 '/Users/bw/Dataset/MIT/images/viz10/000005.png',
 '/Users/bw/Dataset/MIT/images/viz10/000001.png',
 '/Users/bw/Dataset/MIT/images/viz10/000002.png',
 '/Users/bw/Dataset/MIT/images/viz10/000003.png']

# shuffle patch, order is in "order" array

In [35]:
 
# fnames.sort()
flen = len(fnames)

print(flen)


order = np.arange(flen)
# np.random.shuffle(order)

print(order)



pnames_gt = [ fnames[x] for x in order]
pnames_gt[:10]

10
[0 1 2 3 4 5 6 7 8 9]


['/Users/bw/Dataset/MIT/images/viz10/000001.png',
 '/Users/bw/Dataset/MIT/images/viz10/000002.png',
 '/Users/bw/Dataset/MIT/images/viz10/000003.png',
 '/Users/bw/Dataset/MIT/images/viz10/000004.png',
 '/Users/bw/Dataset/MIT/images/viz10/000005.png',
 '/Users/bw/Dataset/MIT/images/viz10/000006.png',
 '/Users/bw/Dataset/MIT/images/viz10/000007.png',
 '/Users/bw/Dataset/MIT/images/viz10/000008.png',
 '/Users/bw/Dataset/MIT/images/viz10/000009.png',
 '/Users/bw/Dataset/MIT/images/viz10/000010.png']

In [36]:
np.random.randint(0, 4, 100)

array([0, 0, 3, 0, 3, 1, 1, 3, 1, 2, 2, 3, 0, 3, 1, 1, 0, 3, 0, 1, 2, 2,
       2, 2, 1, 3, 3, 2, 1, 3, 2, 2, 0, 1, 0, 2, 2, 0, 2, 1, 0, 0, 1, 0,
       2, 3, 0, 0, 2, 2, 3, 3, 2, 1, 0, 0, 0, 0, 3, 0, 1, 1, 0, 0, 3, 1,
       0, 2, 2, 2, 1, 3, 2, 3, 2, 1, 2, 2, 3, 3, 2, 2, 3, 2, 0, 2, 1, 3,
       2, 1, 1, 1, 2, 2, 3, 3, 2, 3, 0, 2])

In [37]:
np.random.normal()

1.4010682567320862

In [38]:
crop_size = 128
cfa_pattern = 2

idx_R = np.tile(
        np.concatenate((np.concatenate((np.ones((cfa_pattern, cfa_pattern)), np.zeros((cfa_pattern, cfa_pattern))), axis=1),
                               np.concatenate((np.zeros((cfa_pattern, cfa_pattern)), np.zeros((cfa_pattern, cfa_pattern))), axis=1)), axis=0),
          (crop_size // 2 // cfa_pattern, crop_size // 2 // cfa_pattern))

idx_B = np.tile(
        np.concatenate((np.concatenate((np.zeros((cfa_pattern, cfa_pattern)), np.zeros((cfa_pattern, cfa_pattern))), axis=1),
                               np.concatenate((np.zeros((cfa_pattern, cfa_pattern)), np.ones((cfa_pattern, cfa_pattern))), axis=1)), axis=0),
          (crop_size // 2 // cfa_pattern, crop_size // 2 // cfa_pattern))

idx_G = np.tile(
        np.concatenate((np.concatenate((np.zeros((cfa_pattern, cfa_pattern)), np.ones((cfa_pattern, cfa_pattern))), axis=1),
                               np.concatenate((np.ones((cfa_pattern, cfa_pattern)), np.zeros((cfa_pattern, cfa_pattern))), axis=1)), axis=0),
          (crop_size // 2 // cfa_pattern, crop_size // 2 // cfa_pattern))

idx_RGB = np.concatenate((idx_R[..., np.newaxis],
                          idx_G[..., np.newaxis],
                          idx_B[..., np.newaxis]), axis=-1)


In [39]:
def make_input_from_gt(gt):
    
    gt_3ch = gt * idx_RGB
    gt_1ch = np.sum(gt_3ch, axis=-1)
    
    inp = gt_1ch*4 + np.random.randint(0,4) # max: 255 -> 1023
    
    # insert bp's
    bp_pos = np.random.normal(size=inp.shape) > 2.33
#     print(bp_pos)
    inp[bp_pos] = 1023
    
    return inp.astype('uint16')
    
    
    
    

In [40]:
def write_tfrecords(file_list_gt, idx_split, dataset_name, output_path):

    ext = file_list_gt[0].split('.')[-1]

    tfrecords_name = '%s_%03d_intp.tfrecords' % (dataset_name, idx_split)
    tfrecords_name = os.path.join(output_path, tfrecords_name)

    if os.path.isfile(tfrecords_name):
        print('OHNO -->. tfrecord (%s) exists' % tfrecords_name)
        #return

    print('tfrecord not exists')
    print('--->', tfrecords_name)

    writer = tf.io.TFRecordWriter(tfrecords_name)

    for index, file_name_gt in enumerate(tqdm.tqdm(file_list_gt)):
#         print('%d / %d' % (index+1, len(file_list)))



        image_gt = Image.open(file_name_gt)


        image_arr_gt = np.array(image_gt).astype(np.uint8)
        # lets make input image
        image_arr_in = make_input_from_gt(image_arr_gt)

        image_byte_gt = image_arr_gt.tobytes()
        image_byte_in = image_arr_in.tobytes()

        example = tf.train.Example(
                features=tf.train.Features(feature={
                    'gt': tf.train.Feature(
                            bytes_list=tf.train.BytesList(value=[image_byte_gt])
                    ),
                    'in': tf.train.Feature(
                            bytes_list=tf.train.BytesList(value=[image_byte_in])
                    )
                })
        )



        writer.write(example.SerializeToString())
    writer.close()

In [41]:
def make_tfrecords(flists, num_splits, dataset_name, output_path='.'):

    flist_gt = flists

    assert len(flist_gt) > 0, f'no file is found'
  

    total_len = len(flist_gt)
    split_len = total_len // num_splits

    print(total_len, split_len)
    start_idx = np.array(list(range(num_splits)) )* split_len
    finish_idx= (np.array((list(range(num_splits)) )) + 1) * split_len

    print(start_idx)
    print(finish_idx)
    finish_idx[-1] = total_len

    # exit()
    # print(start_idx)
    # print(finish_idx)

    for idx_split in range(num_splits):
        print(f'... {idx_split+1} / {num_splits}')
        idx_s = start_idx[idx_split]
        idx_f = finish_idx[idx_split]
        write_tfrecords(flist_gt[idx_s:idx_f], idx_split, dataset_name, output_path)

    print('done make_tfrecords')

In [42]:
output_path = '/Users/bw/Dataset/MIT/tfrecords_pairs'

In [44]:
# make tfrecords
num_of_splits = 1
make_tfrecords(flists=pnames_gt, 
               num_splits=num_of_splits, 
               dataset_name='mit_tetra_viz10_pairs', 
               output_path=output_path)

10 10
[0]
[10]
... 1 / 1
OHNO -->. tfrecord (/Users/bw/Dataset/MIT/tfrecords_pairs/mit_tetra_viz10_pairs_000_intp.tfrecords) exists
tfrecord not exists
---> /Users/bw/Dataset/MIT/tfrecords_pairs/mit_tetra_viz10_pairs_000_intp.tfrecords


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 295.07it/s]

done make_tfrecords


In [ ]:
2480/128, 1920/128

In [ ]:
20*20